In [32]:
import os

import numpy as np
import scipy.misc
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
import tensorflow as tf
from PIL import Image

In [33]:
tf.logging.set_verbosity(tf.logging.INFO)

In [34]:
RED = 0
GREEN = 1
BLUE = 2

In [35]:
def determine_color_from_filename(filename):
    if 'blue' in filename.lower():
        return BLUE
    if 'green' in filename.lower():
        return GREEN
    return RED

In [36]:
def scale_unit_feature(X):
    unit_scaler = MinMaxScaler((0, 1))
    original_size = X.shape
    X = np.reshape(X, (X.shape[0], np.prod(X.shape[1:])))
    X = unit_scaler.fit_transform(X)
    X = np.reshape(X, original_size)
    return X

In [37]:
def convert_to_one_hot(y):
    return OneHotEncoder(3, sparse=False).fit_transform(y).astype('int32')

In [38]:
raw_X = []
raw_y = []
for folder_path, folders, filenames in os.walk('data/input'):
    for filename in filenames:
        filepath = os.path.join(folder_path, filename)
        image = Image.open(filepath).convert('RGB')
        image = np.array(image)
        raw_X.append(image)
        raw_y.append(determine_color_from_filename(filename))

X = np.array(raw_X, dtype='float32')
y = np.array(raw_y, dtype='int32')

In [39]:
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (470, 64, 64, 3)
y.shape:  (470,)


In [40]:
# X = scale_unit_feature(X)
# y = convert_to_one_hot(y)

In [41]:
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (470, 64, 64, 3)
y.shape:  (470,)


In [42]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.2)

In [43]:
print('X_train.shape: ', X_train.shape)
print('X_cv.shape: ', X_cv.shape)
print('y_train.shape: ', y_train.shape)
print('y_cv.shape: ', y_cv.shape)

X_train.shape:  (376, 64, 64, 3)
X_cv.shape:  (94, 64, 64, 3)
y_train.shape:  (376,)
y_cv.shape:  (94,)


Begin Tensorflow Code

In [44]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

In [52]:
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[32],
                                            enable_centered_bias=True,
                                            n_classes=3)


INFO:tensorflow:Using config: {'save_summary_steps': 100, 'keep_checkpoint_max': 5, 'num_ps_replicas': 0, '_job_name': None, 'tf_random_seed': None, 'cluster_spec': None, 'master': '', 'task': 0, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'save_checkpoints_secs': 600, 'keep_checkpoint_every_n_hours': 10000, '_is_chief': True}


In [53]:
classifier.fit(X_train, y_train, batch_size=100, steps=2000)

INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(None), Dimension(64), Dimension(64), Dimension(3)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int32, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=12288, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 106.553, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpl3tivwn4/model.ckpt.
INFO:tensorflow:loss = 0.0, step = 101
INFO:tensorflow:loss = 0.0, step = 201
INFO:tensorflow:loss = 0.0, step = 301
INFO:tensorflow:loss = 0.0, step = 401
INFO:tensorflow:loss = 0.0, step = 501
INFO:tensorflow:loss = 0.0, step = 601
INFO:tensorflow:loss = 0.0, step = 701
INFO:tensorflow:loss = 0.0, step = 801
INFO:tensorflow:loss = 0.0, step = 901
INFO:tensorflow:loss = 0.0, step = 1001


Estimator(params={'activation_fn': <function relu at 0x7f83e810c0d0>, 'weight_column_name': None, 'optimizer': 'Adagrad', 'gradient_clip_norm': None, 'feature_columns': [_RealValuedColumn(column_name='', dimension=12288, default_value=None, dtype=tf.float32, normalizer=None)], 'hidden_units': [32], 'n_classes': 3, 'enable_centered_bias': True, 'num_ps_replicas': 0, 'dropout': None})

In [69]:
y_cv_pred = classifier.predict(X_cv, as_iterable=True)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=12288, default_value=None, dtype=tf.float32, normalizer=None)


In [70]:
y_cv_pred = [i[0] for i in y_cv_pred]
print(y_cv_pred)

INFO:tensorflow:Loading model from checkpoint: /tmp/tmpl3tivwn4/model.ckpt-2000-?????-of-00001.


[1, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 1, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 1, 2, 2, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 2, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1, 2, 2, 1, 1, 0]


In [71]:
print('Accuracy score: ', accuracy_score(y_cv, y_cv_pred))

Accuracy score:  0.968085106383
